In [4]:
from datasets import Dataset
from distilabel.llms import vLLM
from distilabel.pipeline import Pipeline
from dotenv import load_dotenv
from huggingface_hub import hf_hub_download
from vllm import LLM
import os
import re

INFO 03-24 23:08:01 [__init__.py:256] Automatically detected platform cuda.


In [8]:
card_names = [
    'The Fool',
    'The Magician',
    'The High Priestess',
    'The Empress',
    'The Emperor',
    'The Hierophant',
    'The Lovers',
    'The Chariot',
    'Strength',
    'The Hermit',
    'Wheel of Fortune',
    'Justice',
    'The Hanged Man',
    'Death',
    'Temperance',
    'The Devil',
    'The Tower',
    'The Star',
    'The Moon',
    'The Sun',
    'Judgment',
    'The World',
    'Ace of Wands',
    'Two of Wands',
    'Three of Wands',
    'Four of Wands',
    'Five of Wands',
    'Six of Wands',
    'Seven of Wands',
    'Eight of Wands',
    'Nine of Wands',
    'Ten of Wands',
    'Page of Wands',
    'Knight of Wands',
    'Queen of Wands',
    'King of Wands',
    'Ace of Cups',
    'Two of Cups',
    'Three of Cups',
    'Four of Cups',
    'Five of Cups',
    'Six of Cups',
    'Seven of Cups',
    'Eight of Cups',
    'Nine of Cups',
    'Ten of Cups',
    'Page of Cups',
    'Knight of Cups',
    'Queen of Cups',
    'King of Cups',
    'Ace of Swords',
    'Two of Swords',
    'Three of Swords',
    'Four of Swords',
    'Five of Swords',
    'Six of Swords',
    'Seven of Swords',
    'Eight of Swords',
    'Nine of Swords',
    'Ten of Swords',
    'Page of Swords',
    'Knight of Swords',
    'Queen of Swords',
    'King of Swords',
    'Ace of Pentacles',
    'Two of Pentacles',
    'Three of Pentacles',
    'Four of Pentacles',
    'Five of Pentacles',
    'Six of Pentacles',
    'Seven of Pentacles',
    'Eight of Pentacles',
    'Nine of Pentacles',
    'Ten of Pentacles',
    'Page of Pentacles',
    'Knight of Pentacles',
    'Queen of Pentacles',
    'King of Pentacles',
]

topics = {
    "Relationships": ["their mind", "relationship flow", "your move", "challenge", "solution", "evolution"],
    "Wealth": ["current", "future", "opportunity", "resource", "action", "reward"],
    "Career": ["now", "challenge", "future", "skill", "potential", "opportunity"],
    "Self-Growth": ["now", "block", "potential", "mind", "spirit", "guide"],
    "Health": ["past", "present", "future", "challenge", "advice"],
    "Spiritual Guidance": ["past", "present", "future", "challenge", "advice"],
    "Life Purpose": ["past", "present", "future", "challenge", "advice"],
    "Daily Guidance": ["event", "emotion", "reflection"],
    "Weekly Guidance": ["event", "emotion", "reflection"]
}

# Define main topics and guidance topics
main_topics = ["Relationships", "Wealth", "Career", "Self-Growth", "Health", "Spiritual Guidance", "Life Purpose"]
guidance_topics = ["Daily Guidance", "Weekly Guidance"]

# Function to generate prompt for main topics
def generate_main_prompt(card, topic, positional_meanings):
    pos_meaning = random.choice(positional_meanings)
    return f"What does {card} reveal about {pos_meaning} in my {topic}?"

# Function to generate prompts for guidance topics
def generate_guidance_prompts(card, topic):
    prompts = []
    for pos_meaning in topics[topic]:
        prompts.append(f"What {pos_meaning} does {card} suggest for my {topic.lower()}?")
    return prompts

# Generate all prompts
prompts = []
for card in card_names:
    # Generate prompts for main topics (1 prompt per topic per card)
    for topic in main_topics:
        prompt = generate_main_prompt(card, topic, topics[topic])
        prompts.append(prompt)
    # Generate prompts for guidance topics (3 prompts per topic per card)
    for topic in guidance_topics:
        guidance_prompts = generate_guidance_prompts(card, topic)
        prompts.extend(guidance_prompts)

# Output the total number of prompts and a sample
print(f"Total prompts generated: {len(prompts)}")
print("\nSample prompts:")
for prompt in prompts[:5]:
    print(f"- {prompt}")




Total prompts generated: 1014

Sample prompts:
- What does The Fool reveal about relationship flow in my Relationships?
- What does The Fool reveal about resource in my Wealth?
- What does The Fool reveal about future in my Career?
- What does The Fool reveal about block in my Self-Growth?
- What does The Fool reveal about present in my Health?


In [ ]:
from distilabel.models import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline
from distilabel.steps import (
    LoadDataFromHub,
    GroupColumns,
    FormatTextGenerationDPO,
    PreferenceToArgilla,
)

from distilabel.steps.tasks import TextGeneration, UltraFeedback


generate_responses = [
    TextGeneration(
        llm=InferenceEndpointsLLM(
            model_id="sft_awq",
            tokenizer_id="sft_awq",
            generation_kwargs={"max_new_tokens": 512, "temperature": 0.7},
        ),
        pipeline=Pipeline(name="showcase-pipeline"),
    ),
    TextGeneration(
        llm=InferenceEndpointsLLM(
            model_id="sft_qwen",
            tokenizer_id="sft_qwen",
            generation_kwargs={"max_new_tokens": 512, "temperature": 0.7},
        ),
        pipeline=Pipeline(name="showcase-pipeline"),
    ),
]
# for task in generate_responses:
#     task.load()


group_responses = GroupColumns(
    columns=["generation", "model_name"],
    output_columns=["generations", "model_names"],
    pipeline=Pipeline(name="showcase-pipeline"),
)

In [ ]:
from distilabel.steps.tasks import UltraFeedback
from distilabel.models import InferenceEndpointsLLM

evaluate_responses = UltraFeedback(
    aspect="overall-rating",
    # aspect="helpfulness"
    llm=InferenceEndpointsLLM(
        model_id="Qwen/QwQ-32B",
        tokenizer_id="Qwen/QwQ-32B",
        generation_kwargs={"max_new_tokens": 512, "temperature": 0.7},
    ),
    pipeline=Pipeline(name="showcase-pipeline"),
)

format_dpo = FormatTextGenerationDPO(pipeline=Pipeline(name="showcase-pipeline"))
format_dpo.load()

In [ ]:
from distilabel.steps import Step, StepInput
from distilabel.pipeline import Pipeline
from typing import List

class AdjustRatingsStep(Step):
    alpha: float = 0.1  # Configurable hyperparameter

    def process(self, inputs: StepInput) -> List[dict]:
        for row in inputs:
            lengths = [len(gen) for gen in row["generations"]]
            penalties = [self.alpha * length for length in lengths]
            adjusted_ratings = [rating - penalty for rating, penalty in zip(row["ratings"], penalties)]
            row["ratings"] = adjusted_ratings
        return inputs

# Define the pipeline
with Pipeline(name="showcase-pipeline") as pipeline:
    adjust_ratings = AdjustRatingsStep()
    format_dpo = FormatTextGenerationDPO()
    adjust_ratings >> format_dpo  # Connect steps

# Run the pipeline
dataset = [
    {
        "instruction": "What's the capital of Spain?",
        "generations": ["Madrid", "Barcelona"],
        "generation_models": ["Meta-Llama-3-8B-Instruct", "Mixtral-8x7B-Instruct-v0.1"],
        "ratings": [5, 1],
    }
]
result = pipeline.run(inputs=dataset)
print(result)

In [ ]:
# Example dataset
dataset = [
    {
        "instruction": "What's the capital of Spain?",
        "generations": ["Madrid", "Barcelona"],
        "generation_models": ["Meta-Llama-3-8B-Instruct", "Mixtral-8x7B-Instruct-v0.1"],
        "ratings": [5, 1],
    }
]

# Function to adjust ratings based on length
def adjust_ratings(row, alpha=0.1):
    lengths = [len(gen) for gen in row["generations"]]  # Length in characters
    penalties = [alpha * length for length in lengths]
    adjusted_ratings = [rating - penalty for rating, penalty in zip(row["ratings"], penalties)]
    row["ratings"] = adjusted_ratings  # Overwrite original ratings
    return row

# Apply the adjustment
adjusted_dataset = [adjust_ratings(row, alpha=0.1) for row in dataset]

# Process with FormatTextGenerationDPO
format_dpo = FormatTextGenerationDPO(pipeline=Pipeline(name="showcase-pipeline"))
format_dpo.load()
result = next(format_dpo.process(adjusted_dataset))
print(result)